In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
target = ['SalePrice']

In [4]:
df_train = df_train[features + target]

In [5]:
df_train['SalesPrice'] = np.log1p(df_train['SalePrice'])

In [6]:
df_train.describe()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd,SalePrice,SalesPrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,1971.267808,1162.626712,346.992466,1.565068,2.866438,6.517808,180921.195890,12.024057
std,9981.264932,30.202904,386.587738,436.528436,0.550916,0.815778,1.625393,79442.502883,0.399449
min,1300.000000,1872.000000,334.000000,0.000000,0.000000,0.000000,2.000000,34900.000000,10.460271
25%,7553.500000,1954.000000,882.000000,0.000000,1.000000,2.000000,5.000000,129975.000000,11.775105
50%,9478.500000,1973.000000,1087.000000,0.000000,2.000000,3.000000,6.000000,163000.000000,12.001512
75%,11601.500000,2000.000000,1391.250000,728.000000,2.000000,3.000000,7.000000,214000.000000,12.273736
max,215245.000000,2010.000000,4692.000000,2065.000000,3.000000,8.000000,14.000000,755000.000000,13.534474


In [7]:
X, y = df_train[features], df_train.SalePrice

In [8]:
train_X, val_X, train_y, val_y = train_test_split(X, y,
                                                  random_state=42,
                                                  test_size=0.25)

In [9]:
# scaler = StandardScaler()
# 
# scaler.fit(train_X)
# 
# train_X = scaler.transform(train_X)
# val_X = scaler.transform(val_X)

In [10]:
# train_data = lgb.Dataset(train_X, label=train_y)
# test_data = lgb.Dataset(val_X, label=val_y, reference=train_data)

In [11]:
# params = {
#     'objective': 'regression',
#     'metric': 'rmse',
#     'boosting_type': 'gbdt',
#     'num_leaves': 31,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
# }

In [12]:
# num_round = 100
# bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000111 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 792
[LightGBM] [Info] Number of data points in the train set: 1095, number of used features: 7
[LightGBM] [Info] Start training from score 181712.286758


In [13]:
# bst

In [14]:
# Create an instance of the LightGBM Regressor with the RMSE metric. 
model = LGBMRegressor(metric='rmse')

# Train the model using the training data. 
model.fit(train_X, train_y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000147 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 792
[LightGBM] [Info] Number of data points in the train set: 1095, number of used features: 7
[LightGBM] [Info] Start training from score 181712.286758


LGBMRegressor(metric='rmse')

In [15]:
df_test = df_test[['Id'] + features]

In [16]:
y_pred = model.predict(df_test[features], num_iteration=model.best_iteration_)

In [17]:
series = pd.Series(y_pred, name='SalePrice')
df_submit = pd.concat([df_test, series], axis=1)

In [18]:
df_submit.sample(5)

,Id,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd,SalePrice
608,2069,10122,1948,869,0,1,1,3,377548.921818
53,1514,13260,1962,1728,0,2,6,10,377253.891024
83,1544,5000,1925,765,0,1,2,4,370487.114623
531,1992,8220,2000,1008,884,2,3,8,410304.413518
217,1678,14836,2004,2492,0,2,2,8,377253.891024


In [19]:
df_submit.to_csv('submit.csv', columns=['Id', 'SalePrice'], index=False)